In [140]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [141]:
import pandas as pd
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
from pandas.plotting import scatter_matrix
import copy
%matplotlib inline

In [142]:
df = pd.read_csv('/content/drive/MyDrive/dataset_train.csv', delimiter=';')
df.head()

,Obfuscated name,Test_index,Presentation,Question,Data,Class_label_FPG
0,c6c1c29d-0331-46e7-966b-808d34cca6ee,3,2,10,"[-24, 51, 75, 51, 14, -13, -25, -45, -77, -76,...",1
1,0fbd87a7-f107-4e0f-aef2-a14ecfe658e6,2,1,11,"[-305, -320, -278, -267, -308, -375, -467, -53...",1
2,23a36ef6-45be-4ed9-9c4d-0169283054b5,2,2,6,"[60, 37, 39, 69, 60, 30, -7, -37, -60, -71, -8...",1
3,c5ba273c-c85e-4ab0-94a8-647f4589989d,1,3,7,"[65, 48, 30, 18, 9, 0, -18, -35, -1, 83, 119, ...",1
4,c291a7ab-817d-497d-8cfe-86df504a665f,4,1,4,"[116, 95, 80, 78, 78, 73, 63, 47, 45, 94, 119,...",1


In [143]:
# df = df[df['Obfuscated name']=='23a36ef6-45be-4ed9-9c4d-0169283054b5']
# df.pop('Obfuscated name')
# df.pop('Test_index')
# df.pop('Presentation')
# df.pop('Question')
# data = []
# for i,row in df.iterrows():
#   data.append(np.fromstring(row[0][1:-1],sep=','))
# data = np.array(data)
# data
# df

In [144]:
# from matplotlib import colors
# data0=[]
# data1=[]
# data2 =[]
# sum = 0
# for i,row in df.iterrows():
#   if row['Class_label_FPG']==0:
#     data0.append(abs(max(np.fromstring(row['Data'][1:-1], sep=','))-min(np.fromstring(row['Data'][1:-1], sep=','))))
#     sum+=abs(max(np.fromstring(row['Data'][1:-1], sep=','))-min(np.fromstring(row['Data'][1:-1], sep=',')))
#   if row['Class_label_FPG']==1:
#     data1.append(abs(max(np.fromstring(row['Data'][1:-1], sep=','))-min(np.fromstring(row['Data'][1:-1], sep=','))))
#     sum+=abs(max(np.fromstring(row['Data'][1:-1], sep=','))-min(np.fromstring(row['Data'][1:-1], sep=',')))
#   if row['Class_label_FPG']==2:
#     data2.append(abs(max(np.fromstring(row['Data'][1:-1], sep=','))-min(np.fromstring(row['Data'][1:-1], sep=','))))
#     sum+=abs(max(np.fromstring(row['Data'][1:-1], sep=','))-min(np.fromstring(row['Data'][1:-1], sep=',')))
# sr = sum/len(df)
# dd=[]
# for i in data2:
#   dd.append(sr)



# plt.plot(data0, 'r')
# plt.plot(data1, 'g')
# plt.plot(data2, 'y')
# plt.plot(dd, 'b')


In [145]:
df = df.sort_values(by='Obfuscated name')

In [146]:
df.pop('Test_index')
df.pop('Presentation')
df.pop('Question')

551       6
303       9
24980     5
2784      9
23227    11
         ..
21942     2
6491      6
27435     1
9694     12
21782     7
Name: Question, Length: 28713, dtype: int64

Создание столбца дата в формате np.array

In [147]:
arr = []
for i,row in df.iterrows():
  arr.append(np.fromstring(row['Data'][1:-1], sep=','))
df['new_data'] = arr

Подсчет фичи по одной реакции человека

In [148]:
arr_max_min = []
arr_start_end = []
arr_sum = []
arr_median = []
for i,row in df.iterrows():
  arr_max_min.append(abs(max(row['new_data'])-min(row['new_data'])))
  arr_start_end.append(abs(row['new_data'][0]-row['new_data'][-1]))
  arr_sum.append(abs(row['new_data'].sum()))
  arr_median.append(abs(np.median(row['new_data'])))
df['max_min'] = arr_max_min
df['start_end'] = arr_start_end
df['sum'] = arr_sum
df['median'] = arr_median

Подсчет средней фичи по всем реациям одного человека и связывание этого путем получение модуля разности с фичей по одной реакции 

In [149]:
for i in df['Obfuscated name'].unique():
  dff = df[df['Obfuscated name']==i]
  arr_mean_max_min = []
  arr_mean_start_end = []
  arr_mean_sum = []
  arr_mean_median = []
  mean_max_min = dff.describe()['max_min']['mean']
  mean_start_end = dff.describe()['start_end']['mean'] 
  mean_sum = dff.describe()['sum']['mean'] 
  mean_median= abs(dff.describe()['median']['mean'])
  for i in arr_max_min:
    arr_mean_max_min.append(abs(mean_max_min-i))
  for i in arr_start_end:
    arr_mean_start_end.append(abs(mean_start_end-i))
  for i in arr_sum:
    arr_mean_sum.append(abs(mean_sum-i))
  for i in arr_median:
    arr_mean_median.append(abs(mean_median-i))

df['mean_max_min'] = arr_mean_max_min
df['mean_start_end'] = arr_mean_start_end
df['mean_sum'] = arr_mean_sum
df['mean_median'] = arr_mean_median

df.head()

,Obfuscated name,Data,Class_label_FPG,new_data,max_min,start_end,sum,median,mean_max_min,mean_start_end,mean_sum,mean_median
551,001945e6-1756-4745-bd86-c016f6ba6e6b,"[-52, -156, -245, -342, -424, -436, -45, 493, ...",2,"[-52.0, -156.0, -245.0, -342.0, -424.0, -436.0...",2158.0,280.0,161771.0,750.5,1551.517241,112.862069,155881.0,729.87931
303,001945e6-1756-4745-bd86-c016f6ba6e6b,"[237, 221, 197, 135, 38, -63, -144, -211, -271...",1,"[237.0, 221.0, 197.0, 135.0, 38.0, -63.0, -144...",1731.0,486.0,15945.0,97.5,1124.517241,318.862069,10055.0,76.87931
24980,001945e6-1756-4745-bd86-c016f6ba6e6b,"[326, 216, 137, 80, 16, -69, -127, -118, 81, 4...",0,"[326.0, 216.0, 137.0, 80.0, 16.0, -69.0, -127....",1709.0,530.0,46512.0,156.0,1102.517241,362.862069,40622.0,135.37931
2784,001945e6-1756-4745-bd86-c016f6ba6e6b,"[443, 324, 203, 123, 35, -30, -121, -193, -203...",2,"[443.0, 324.0, 203.0, 123.0, 35.0, -30.0, -121...",3156.0,1355.0,111691.0,680.0,2549.517241,1187.862069,105801.0,659.37931
23227,001945e6-1756-4745-bd86-c016f6ba6e6b,"[173, 122, 85, 49, -7, -75, -118, -25, 260, 56...",1,"[173.0, 122.0, 85.0, 49.0, -7.0, -75.0, -118.0...",1663.0,484.0,35218.0,120.0,1056.517241,316.862069,29328.0,99.37931


Разделение на трейн и тест

In [150]:
df_train = df.iloc[0:int(len(df)*0.75)]
df_test = df.iloc[int(len(df)*0.75):28713]

Создание массивов для обучения и предсказания

In [151]:
from sklearn import preprocessing 

data_train = [] 
data_test = []
for i,row in df_train.iterrows():
  arr = []
  arr.append(row['mean_max_min'])
  arr.append(row['mean_start_end'])
  arr.append(row['mean_sum'])
  arr.append(row['mean_median'])
  arr = np.array(arr)
  data_train.append(arr)
data_train = np.array(data_train)
for i,row in df_test.iterrows():
  arr = []
  arr.append(row['mean_max_min'])
  arr.append(row['mean_start_end'])
  arr.append(row['mean_sum'])
  arr.append(row['mean_median'])
  arr = np.array(arr)
  data_test.append(arr)
data_test = np.array(data_test)

scaler = preprocessing.StandardScaler().fit(data_test)
data_test = scaler.transform(data_test)
scaler = preprocessing.StandardScaler().fit(data_train)
data_train = scaler.transform(data_train)

Обучение и предсказания

In [152]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle




smote = SMOTE()
X_train, y_train = smote.fit_resample(data_train, df_train['Class_label_FPG'])
# X_train = data_train
# y_train = df_train['Class_label_FPG']

# X_train = data_train
X_test = data_test
# y_train = df_train['Class_label_FPG']
y_test = df_test['Class_label_FPG']

knn = KNeighborsClassifier(n_neighbors = 3,algorithm='ball_tree').fit(X_train, y_train)
# rbf = svm.SVC(kernel='rbf', gamma=1, C=50).fit(X_train, y_train)
gnb = GaussianNB().fit(X_train,y_train)
logreg = LogisticRegression(solver = 'lbfgs',multi_class='auto').fit(X_train,y_train)
dt = DecisionTreeClassifier().fit(X_train,y_train)
dc = DummyClassifier(strategy='stratified').fit(X_train,y_train)
rfc = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42).fit(X_train, y_train)


knn_pred = knn.predict(X_test) 
# rbf_pred = rbf.predict(X_test)
gnb_pred = gnb.predict(X_test)
logreg_pred = logreg.predict(X_test)
dt_pred = dt.predict(X_test)
dc_pred = dc.predict(X_test)
rfc_pred = rfc.predict(X_test)

# pkl_filename = "model.pkl"
# with open(pkl_filename, 'wb') as file:
#   pickle.dump(gnb_pred, file)

y_test = np.array(y_test)
max_error=0
for i in y_test:
  if i == 0 or i == 2:
    max_error+=2
  else:
    max_error+=1

b = 0
for i in range(len(y_test)):
  b+=abs(knn_pred[i]-y_test[i])
print('knn: ',1 - b/max_error)
# b=0
# for i in range(len(y_test)):
#   b+=abs(rbf_pred[i]-y_test[i])
# print('rbf: ',1 - b/max_error)
b = 0
for i in range(len(y_test)):
  b+=abs(gnb_pred[i]-y_test[i])
print('gnb: ',1 - b/max_error)
b = 0
for i in range(len(y_test)):
  b+=abs(logreg_pred[i]-y_test[i])
print('logreg: ',1 - b/max_error)
b = 0
for i in range(len(y_test)):
  b+=abs(dt_pred[i]-y_test[i])
print('dt: ',1 - b/max_error)
b = 0
for i in range(len(y_test)):
  b+=abs(dc_pred[i]-y_test[i])
print('dc: ',1 - b/max_error)
b = 0
for i in range(len(y_test)):
  b+=abs(rfc_pred[i]-y_test[i])
print('rfc: ',1 - b/max_error)

knn:  0.44859377910225373
gnb:  0.6496554293164463
logreg:  0.43285528031290743
dt:  0.48631029986962193
dc:  0.4361147327249022
rfc:  0.4986962190352021


In [153]:
b = 0
for i in range(len(y_test)):
  b+=abs(1-y_test[i])
print('rfc: ',1 - b/max_error)

rfc:  0.668560253306016


Тест

In [154]:
dt = pd.read_csv('/content/drive/MyDrive/test.csv', delimiter=',')
dt.head()
dt = dt.sort_values(by='Obfuscated name')
dt.pop('Test_index')
dt.pop('Presentation')
dt.pop('Question')

arr = []
for i,row in dt.iterrows():
  arr.append(np.fromstring(row['Data'][1:-1], sep=','))
dt['new_data'] = arr

arr_max_min = []
arr_start_end = []
arr_sum = []
arr_median = []
for i,row in dt.iterrows():
  arr_max_min.append(abs(max(row['new_data'])-min(row['new_data'])))
  arr_start_end.append(abs(row['new_data'][0]-row['new_data'][-1]))
  arr_sum.append(abs(row['new_data'].sum()))
  arr_median.append(abs(np.median(row['new_data'])))
dt['max_min'] = arr_max_min
dt['start_end'] = arr_start_end
dt['sum'] = arr_sum
dt['median'] = arr_median

for i in dt['Obfuscated name'].unique():
  dtt = dt[dt['Obfuscated name']==i]
  arr_mean_max_min = []
  arr_mean_start_end = []
  arr_mean_sum = []
  arr_mean_median = []
  mean_max_min = dtt.describe()['max_min']['mean']
  mean_start_end = dtt.describe()['start_end']['mean'] 
  mean_sum = dtt.describe()['sum']['mean'] 
  mean_median=abs(dtt.describe()['median']['mean'])
  for i in arr_max_min:
    arr_mean_max_min.append(abs(mean_max_min-i))
  for i in arr_start_end:
    arr_mean_start_end.append(abs(mean_start_end-i))
  for i in arr_sum:
    arr_mean_sum.append(abs(mean_sum-i))
  for i in arr_median:
    arr_mean_median.append(abs(mean_median-i))

dt['mean_max_min'] = arr_mean_max_min
dt['mean_start_end'] = arr_mean_start_end
dt['mean_sum'] = arr_mean_sum
dt['mean_median'] = arr_mean_sum

data = [] 

for i,row in dt.iterrows():
  arr = []
  arr.append(row['mean_max_min'])
  arr.append(row['mean_start_end'])
  arr.append(row['mean_sum'])
  arr.append(row['mean_median'])
  arr = np.array(arr)
  data.append(arr)
data = np.array(data)


scaler = preprocessing.StandardScaler().fit(data)
data = scaler.transform(data)

X = data

gnb_pred = gnb.predict(X)

In [155]:
count = 0
for i in gnb_pred:
  if i==1:
    count+=1
count

1493

In [156]:
dt['class']=gnb_pred
dt

,id,Obfuscated name,Data,new_data,max_min,start_end,sum,median,mean_max_min,mean_start_end,mean_sum,mean_median,class
336,336,04ec73d4-3906-4e4e-a8b8-d5eec8fd7eb2,"[4345, 4641, 5395, 5731, 5992, 6173, 6132, 606...","[4345.0, 4641.0, 5395.0, 5731.0, 5992.0, 6173....",3906.0,808.0,1175017.0,4901.0,148.944444,219.638889,369255.166667,369255.166667,1
304,304,04ec73d4-3906-4e4e-a8b8-d5eec8fd7eb2,"[4670, 4472, 4261, 4066, 3875, 3671, 3420, 322...","[4670.0, 4472.0, 4261.0, 4066.0, 3875.0, 3671....",3896.0,26.0,1034617.0,4258.0,158.944444,1001.638889,228855.166667,228855.166667,1
305,305,04ec73d4-3906-4e4e-a8b8-d5eec8fd7eb2,"[3771, 3797, 3724, 3599, 3515, 3437, 3280, 313...","[3771.0, 3797.0, 3724.0, 3599.0, 3515.0, 3437....",4449.0,944.0,1188599.0,5012.5,394.055556,83.638889,382837.166667,382837.166667,1
306,306,04ec73d4-3906-4e4e-a8b8-d5eec8fd7eb2,"[5102, 4998, 4882, 4776, 4640, 4421, 4244, 405...","[5102.0, 4998.0, 4882.0, 4776.0, 4640.0, 4421....",5381.0,153.0,1134730.0,4640.0,1326.055556,874.638889,328968.166667,328968.166667,1
307,307,04ec73d4-3906-4e4e-a8b8-d5eec8fd7eb2,"[4845, 4741, 4593, 4446, 4248, 4019, 3709, 342...","[4845.0, 4741.0, 4593.0, 4446.0, 4248.0, 4019....",4503.0,166.0,1173408.0,4860.0,448.055556,861.638889,367646.166667,367646.166667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,2081,fe34715a-f38e-4c2c-bae3-fbda6ad1a76a,"[2857, 2769, 2944, 3421, 3741, 3668, 3458, 333...","[2857.0, 2769.0, 2944.0, 3421.0, 3741.0, 3668....",2627.0,1185.0,659247.0,2686.0,1427.944444,157.361111,146514.833333,146514.833333,1
2080,2080,fe34715a-f38e-4c2c-bae3-fbda6ad1a76a,"[4883, 4833, 4718, 4691, 4619, 4497, 4331, 419...","[4883.0, 4833.0, 4718.0, 4691.0, 4619.0, 4497....",3035.0,1980.0,866505.0,3676.0,1019.944444,952.361111,60743.166667,60743.166667,0
2079,2079,fe34715a-f38e-4c2c-bae3-fbda6ad1a76a,"[4660, 4622, 4605, 4520, 4325, 4127, 4019, 389...","[4660.0, 4622.0, 4605.0, 4520.0, 4325.0, 4127....",2748.0,1809.0,817305.0,3208.5,1306.944444,781.361111,11543.166667,11543.166667,0
2077,2077,fe34715a-f38e-4c2c-bae3-fbda6ad1a76a,"[5521, 5274, 5085, 4873, 4703, 4542, 4826, 551...","[5521.0, 5274.0, 5085.0, 4873.0, 4703.0, 4542....",5225.0,1760.0,832162.0,3748.5,1170.055556,732.361111,26400.166667,26400.166667,0


In [157]:
dm = pd.read_csv('/content/drive/MyDrive/test.csv', delimiter=',')
classes = []
for i1, row in dm.iterrows():
  c = dt[dt['id']==row['id']]
  classes.append(c['class'].values[0])
dm['Class'] = classes
dm.pop('Obfuscated name')
dm.pop('Test_index')
dm.pop('Presentation')
dm.pop('Question')
dm.pop('Data')
dm.to_csv('Надо_подумать.csv', index=False)

In [158]:
dm.to_csv('Надо_подумать.csv', index=False)